In [37]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [38]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from scipy.stats import randint
from sklearn.model_selection import cross_val_predict

In [39]:
import pandas as pd
import numpy as np

In [40]:
df_data = pd.read_csv("./assay_data_filtered_88_inchi.csv") 
assay_list= df_data.columns.to_list()[1:-1]
df_data = df_data.drop(columns= ["smiles_r"])
df_data

,InChICode_standardised,588458,588334,2642,2156,2330,2216,743015,504444,894,...,652104,720579,720533,720542,720580,720504,720532,1159524,1117304,1117305
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [41]:
len(assay_list)

88

In [42]:
CP_features = pd.read_csv("./CP_filtered_9876.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_AreaShape_MedianRadius,Cells_Correlation_Correlation_DNA_AGP,Cells_Correlation_Correlation_ER_AGP,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,InChICode_standardised
0,0.963798,0.159010,0.047354,-0.137963,0.216676,0.144280,0.113497,-0.117675,-0.079211,-0.206469,...,-0.245524,-0.268078,-0.123740,0.122226,0.402861,-0.206448,-0.158874,-0.121066,0.185686,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...
1,1.210207,0.367478,0.263613,0.083119,-0.181781,-0.118582,-0.041744,0.069925,0.043344,0.016976,...,-0.125642,0.168251,-0.116379,0.316108,0.299225,-0.173420,0.057710,0.010376,0.272074,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...
2,0.581696,0.125987,0.092521,-0.021064,0.054373,-0.037538,0.053001,0.018777,-0.053614,0.075178,...,-0.059526,0.071001,-0.029977,-1.310799,-1.847892,-0.000630,0.055282,0.028152,-0.610017,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...
3,0.248304,-0.014260,-0.018056,-0.037718,0.063135,0.032092,-0.014411,-0.009945,-0.000591,-0.023499,...,-0.554508,-0.357952,-0.232800,-0.197858,-0.499390,-0.065841,-0.123535,-0.071141,-0.109201,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...
4,-2.382596,0.342042,0.183357,-0.814742,1.361776,3.826898,3.387335,-0.507087,-0.559135,-0.432035,...,-2.017392,1.497684,0.264454,-0.562605,1.086744,0.311285,0.727071,0.874987,0.189458,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,-0.000341,-0.022052,-0.049354,-0.077226,0.036142,0.035560,0.034507,-0.021980,-0.036769,-0.026607,...,-0.154942,-0.208285,0.094947,-0.064585,-0.062841,-0.050082,-0.050364,0.009238,-0.025615,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...
9872,0.148294,0.043186,0.001665,-0.007765,-0.000027,-0.011346,0.075195,0.006984,-0.059737,-0.038276,...,-0.421050,-0.162980,-0.191575,-0.036555,-0.244200,-0.052731,-0.053152,-0.102096,0.074795,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...
9873,0.043964,0.063290,0.016303,-0.056327,0.025226,0.010598,0.121048,-0.004245,-0.069040,-0.121069,...,0.805582,0.352569,-0.237808,0.390295,0.707784,0.231027,0.102048,-0.001059,0.298214,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...
9874,-0.080926,-0.094140,-0.105359,0.045901,-0.024562,0.026936,-0.022409,-0.024922,0.042600,-0.009710,...,-0.200248,-0.237660,-0.151925,-0.112267,-0.158519,-0.020584,-0.080274,-0.091421,-0.034917,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...


In [43]:
mfp_features = pd.read_csv("./Mfp_filtered_9876.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["InChICode_standardised", "fp"]]
mfp_features

,InChICode_standardised,fp
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,"[1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
assay_data= pd.merge(df_data, CP_features)
assay_data= pd.merge(assay_data, mfp_features)
assay_data

,InChICode_standardised,588458,588334,2642,2156,2330,2216,743015,504444,894,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.245524,-0.268078,-0.123740,0.122226,0.402861,-0.206448,-0.158874,-0.121066,0.185686,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.125642,0.168251,-0.116379,0.316108,0.299225,-0.173420,0.057710,0.010376,0.272074,"[1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,-0.059526,0.071001,-0.029977,-1.310799,-1.847892,-0.000630,0.055282,0.028152,-0.610017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,-0.554508,-0.357952,-0.232800,-0.197858,-0.499390,-0.065841,-0.123535,-0.071141,-0.109201,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,-2.017392,1.497684,0.264454,-0.562605,1.086744,0.311285,0.727071,0.874987,0.189458,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.154942,-0.208285,0.094947,-0.064585,-0.062841,-0.050082,-0.050364,0.009238,-0.025615,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.421050,-0.162980,-0.191575,-0.036555,-0.244200,-0.052731,-0.053152,-0.102096,0.074795,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.805582,0.352569,-0.237808,0.390295,0.707784,0.231027,0.102048,-0.001059,0.298214,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.200248,-0.237660,-0.151925,-0.112267,-0.158519,-0.020584,-0.080274,-0.091421,-0.034917,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [45]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)
        
        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay outercv#", i+1)    
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        'max_features': randint(40, 100),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay outercv#", i+1)  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val, in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y):
            detail_list_batch_train.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val, "Training"])
    
    return detail_list_batch_train

In [46]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts


def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        
        #Prepare features
        train_x_fp, test_x_fp, train_y, test_y = list(train.fp), list(test.fp), train[assay], test[assay]
        col_list=[]
      
        train_x_cp=train[CP_features_list].to_numpy()
        test_x_cp=test[CP_features_list].to_numpy()
        
        #MFP
        print("Mfp_Model_Running for assay")    
        inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
         
        param_dist_grid = {
        'max_depth': [10,15,20],
        'max_features': randint(40, 100),
        'min_samples_leaf': randint(2, 15),
        'min_samples_split': randint(2, 15),
        'n_estimators':[100, 300, 400, 500],
        'bootstrap': [True, False],
        'oob_score': [False],
        'random_state': [42],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'class_weight' : [None, 'balanced']
        }
        
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)

        rsh.fit(train_x_fp, train_y)

        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)                      
        classifier.fit(train_x_fp, train_y)
        
        #Threshold Balancing
        cross_val_prob_fp = cross_val_predict(rsh.best_estimator_, train_x_fp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_fp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_fp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_fp))


        fp_proba = classifier.predict_proba(test_x_fp)[:,1]
        #fp_pred = classifier.predict(test_x_fp) 
        fp_pred  = [ 1 if x>best_thresh_fp  else 0 for x in fp_proba ] 
    
        #Cell Painting
        
        print("CP_Model_Running for assay")  
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
      
        rf = RandomForestClassifier(n_jobs=-1)
        rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, random_state=42, n_jobs=40, verbose=0,  cv = inner_cv)
        
        
        rsh.fit(train_x_cp, train_y)
        
        print("Tuned Mode: ", rsh.best_params_)
        classifier = rsh.best_estimator_    
        
        #classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1)
        classifier.fit(train_x_cp, train_y)
        
        #Threshold Balancing
        cross_val_prob_cp = cross_val_predict(rsh.best_estimator_, train_x_cp, train_y, cv=inner_cv, method='predict_proba')[:, 1]
        # calculate roc curves
        fpr, tpr, thresholds = roc_curve(train_y, cross_val_prob_cp)
        # get the best threshold
        J = tpr - fpr
        ix = argmax(J)
        best_thresh_cp = thresholds[ix]
        print('Best Threshold=%f' % (best_thresh_cp))


        CP_proba = classifier.predict_proba(test_x_cp)[:,1]
        #CP_pred = classifier.predict(test_x_cp) 
        CP_pred  = [ 1 if x>best_thresh_cp  else 0 for x in CP_proba ] 
        
        fpr, tpr, _ = roc_curve(test_y, CP_proba)

        #combine results
        for InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, true_val,  in zip(test.InChICode_standardised, fp_proba, fp_pred, CP_proba, CP_pred, test_y):
            detail_list_batch_test.append([assay, InChICode_standardised, fp_proba, fp_pred,best_thresh_fp, CP_proba, CP_pred, best_thresh_cp, true_val,  "HeldOut"])
    
        return detail_list_batch_test

In [48]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= assay_data[pd.notnull(assay_data[assay])]

    print("Total Compounds ", len(df_assay) )
    inactives_n=len(df_assay[df_assay[assay] == 0])
    print("Inactive Compounds ", inactives_n)
    actives_n=len(df_assay[df_assay[assay] == 1])
    print("Active Compounds ", actives_n)
    
    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
    
    

  0%|                                                    | 0/88 [00:00<?, ?it/s]

Assay  588458
Total Compounds  448
Inactive Compounds  336
Active Compounds  112
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.227732
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.291203
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'max_features': 83, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_st

  1%|▍                                         | 1/88 [01:34<2:16:34, 94.19s/it]

Best Threshold=0.304308
Assay  588334
Total Compounds  528
Inactive Compounds  396
Active Compounds  132
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 83, 'min_samples_leaf': 9, 'min_samples_split': 9, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.258953
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 83, 'min_samples_leaf': 9, 'min_samples_split': 9, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.273340
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 83, 'min_samples_leaf': 9, 'min_samples_split': 9, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False

  2%|▉                                        | 2/88 [03:41<2:43:19, 113.95s/it]

...

Best Threshold=0.359881
Assay  1117304
Total Compounds  386
Inactive Compounds  260
Active Compounds  126
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'max_features': 83, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.389468
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 15, 'max_features': 83, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.216308
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 82, 'min_samples_leaf': 9, 'min_samples_split': 14, 'n_estimators': 100, 'n_jobs': -1, 'o

 99%|████████████████████████████████████████▌| 87/88 [2:58:48<01:27, 87.07s/it]

Best Threshold=0.449416
Assay  1117305
Total Compounds  359
Inactive Compounds  247
Active Compounds  112
Perform nested cross validation on training set
Mfp_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.385943
CP_Model_Running for assay outercv# 1
Tuned Mode:  {'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 99, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': 42}
Best Threshold=0.440213
Mfp_Model_Running for assay outercv# 2
Tuned Mode:  {'bootstrap': False, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'max_features': 99, 'min_samples_leaf': 8, 'min_samples_split': 13, 'n_estimators': 100, 'n_jobs': -1, 'oob

100%|████████████████████████████████████████| 88/88 [3:00:14<00:00, 122.89s/it]

Best Threshold=0.264151


In [49]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","InChICode_standardised","fp_proba", "fp_pred", "fp_threshold", "CP_proba", 
                                                     "CP_pred","CP_threshold","true", "Data"])
list_of_lists_df["MFP_Correct"] = list_of_lists_df["fp_pred"]==list_of_lists_df["true"]
list_of_lists_df["CP_Correct"] = list_of_lists_df["CP_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,True,False
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,False,False
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,True,True
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,False,True
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True


In [50]:
len(list_of_lists_df[list_of_lists_df["Data"]=="Training"])

40124

In [51]:
len(list_of_lists_df[list_of_lists_df["Data"]=="HeldOut"])

10072

In [52]:
#Probability balancing to thresholds

new_results= pd.DataFrame()
new_new_results = pd.DataFrame()

for t in list(list_of_lists_df["fp_threshold"].unique()):
        
        detail_list_fold=list_of_lists_df[list_of_lists_df["fp_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["fp_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["fp_proba_scaled"]=k_list
        new_results=pd.concat([new_results, detail_list_fold])
       
for t in list(new_results["CP_threshold"].unique()):
        
        detail_list_fold=new_results[new_results["CP_threshold"]==t]
        print(len(detail_list_fold["fp_proba"].values), "Compounds in Fold with Threshold: ", np.round(t, 3))
        #print(detail_list_fold["fp_proba"].values)
        k_list=[]
        for p in detail_list_fold["CP_proba"].values:
            if p<=t:
                #print(p)
                k=p*0.5/t

            else:
                #print(p)
                k=1-((1-p)*0.5)/(1-t)

            k_list.append(k)
        #print(k_list)
        detail_list_fold["CP_proba_scaled"]=k_list    
        
        new_new_results=pd.concat([new_new_results, detail_list_fold])

new_new_results

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


72 Compounds in Fold with Threshold:  0.228
72 Compounds in Fold with Threshold:  0.212
72 Compounds in Fold with Threshold:  0.308
71 Compounds in Fold with Threshold:  0.213
71 Compounds in Fold with Threshold:  0.444
90 Compounds in Fold with Threshold:  0.31
85 Compounds in Fold with Threshold:  0.259
85 Compounds in Fold with Threshold:  0.275
84 Compounds in Fold with Threshold:  0.332
84 Compounds in Fold with Threshold:  0.27
84 Compounds in Fold with Threshold:  0.3
106 Compounds in Fold with Threshold:  0.236
123 Compounds in Fold with Threshold:  0.394
122 Compounds in Fold with Threshold:  0.226
122 Compounds in Fold with Threshold:  0.252
122 Compounds in Fold with Threshold:  0.33
122 Compounds in Fold with Threshold:  0.246
153 Compounds in Fold with Threshold:  0.256
91 Compounds in Fold with Threshold:  0.215
91 Compounds in Fold with Threshold:  0.294
91 Compounds in Fold with Threshold:  0.447
91 Compounds in Fold with Threshold:  0.304
90 Compounds in Fold with Thre

75 Compounds in Fold with Threshold:  0.17
75 Compounds in Fold with Threshold:  0.454
75 Compounds in Fold with Threshold:  0.319
75 Compounds in Fold with Threshold:  0.364
74 Compounds in Fold with Threshold:  0.34
94 Compounds in Fold with Threshold:  0.233
20 Compounds in Fold with Threshold:  0.088
19 Compounds in Fold with Threshold:  0.508
19 Compounds in Fold with Threshold:  0.085
19 Compounds in Fold with Threshold:  0.503
19 Compounds in Fold with Threshold:  1.579
25 Compounds in Fold with Threshold:  0.49
21 Compounds in Fold with Threshold:  0.691
21 Compounds in Fold with Threshold:  0.423
21 Compounds in Fold with Threshold:  0.443
20 Compounds in Fold with Threshold:  0.29
20 Compounds in Fold with Threshold:  0.568
26 Compounds in Fold with Threshold:  0.506
25 Compounds in Fold with Threshold:  0.434
24 Compounds in Fold with Threshold:  0.601
24 Compounds in Fold with Threshold:  0.455
24 Compounds in Fold with Threshold:  0.407
24 Compounds in Fold with Threshold:

17 Compounds in Fold with Threshold:  0.334
17 Compounds in Fold with Threshold:  0.449
17 Compounds in Fold with Threshold:  0.486
16 Compounds in Fold with Threshold:  0.548
16 Compounds in Fold with Threshold:  0.368
21 Compounds in Fold with Threshold:  0.212
114 Compounds in Fold with Threshold:  0.716
114 Compounds in Fold with Threshold:  0.682
114 Compounds in Fold with Threshold:  0.764
113 Compounds in Fold with Threshold:  0.792
113 Compounds in Fold with Threshold:  0.762
142 Compounds in Fold with Threshold:  0.781
73 Compounds in Fold with Threshold:  0.761
72 Compounds in Fold with Threshold:  0.919
72 Compounds in Fold with Threshold:  0.617
72 Compounds in Fold with Threshold:  0.871
72 Compounds in Fold with Threshold:  0.879
91 Compounds in Fold with Threshold:  0.623
25 Compounds in Fold with Threshold:  0.619
24 Compounds in Fold with Threshold:  0.484
24 Compounds in Fold with Threshold:  0.551
24 Compounds in Fold with Threshold:  0.495
24 Compounds in Fold with 

79 Compounds in Fold with Threshold:  0.348
78 Compounds in Fold with Threshold:  0.203
78 Compounds in Fold with Threshold:  0.307
99 Compounds in Fold with Threshold:  0.243
117 Compounds in Fold with Threshold:  0.289
117 Compounds in Fold with Threshold:  0.161
116 Compounds in Fold with Threshold:  0.357
116 Compounds in Fold with Threshold:  0.175
116 Compounds in Fold with Threshold:  0.312
146 Compounds in Fold with Threshold:  0.18
58 Compounds in Fold with Threshold:  0.453
58 Compounds in Fold with Threshold:  0.647
58 Compounds in Fold with Threshold:  0.565
58 Compounds in Fold with Threshold:  0.494
57 Compounds in Fold with Threshold:  0.449
73 Compounds in Fold with Threshold:  0.494
547 Compounds in Fold with Threshold:  0.262
547 Compounds in Fold with Threshold:  0.228
546 Compounds in Fold with Threshold:  0.273
546 Compounds in Fold with Threshold:  0.272
546 Compounds in Fold with Threshold:  0.251
684 Compounds in Fold with Threshold:  0.253
66 Compounds in Fold 

18 Compounds in Fold with Threshold:  0.541
18 Compounds in Fold with Threshold:  0.659
23 Compounds in Fold with Threshold:  0.582
20 Compounds in Fold with Threshold:  0.424
20 Compounds in Fold with Threshold:  0.391
20 Compounds in Fold with Threshold:  0.386
20 Compounds in Fold with Threshold:  0.64
20 Compounds in Fold with Threshold:  0.698
26 Compounds in Fold with Threshold:  0.31
68 Compounds in Fold with Threshold:  0.238
68 Compounds in Fold with Threshold:  0.251
68 Compounds in Fold with Threshold:  0.201
68 Compounds in Fold with Threshold:  0.432
67 Compounds in Fold with Threshold:  0.264
85 Compounds in Fold with Threshold:  0.238
422 Compounds in Fold with Threshold:  0.249
421 Compounds in Fold with Threshold:  0.47
421 Compounds in Fold with Threshold:  0.476
421 Compounds in Fold with Threshold:  0.385
421 Compounds in Fold with Threshold:  0.432
527 Compounds in Fold with Threshold:  0.333
61 Compounds in Fold with Threshold:  0.485
61 Compounds in Fold with Thr

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.072320,0,0.227732,0.291514,1,0.291203,0.0,Training,True,False,0.158783,0.500219
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.241077,1,0.227732,0.334301,1,0.291203,0.0,Training,False,False,0.508641,0.530402
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.119513,0,0.227732,0.186209,0,0.291203,0.0,Training,True,True,0.262399,0.319723
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.045993,0,0.227732,0.472894,1,0.291203,1.0,Training,False,True,0.100980,0.628169
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.269438,1,0.227732,0.168755,0,0.291203,0.0,Training,False,True,0.527003,0.289754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.703995,1,0.457779,0.556806,1,0.264151,0.0,HeldOut,False,False,0.727044,0.698855
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.196017,0,0.457779,0.210436,0,0.264151,0.0,HeldOut,True,True,0.214095,0.398324
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.202912,0,0.457779,0.487726,1,0.264151,1.0,HeldOut,False,True,0.221627,0.651916
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.149597,0,0.457779,0.238125,0,0.264151,0.0,HeldOut,True,True,0.163395,0.450736


In [53]:
new_new_results.to_csv("Predictions_train_heldout_scaled_prob_all_assays.csv", index=False)